# Bayesian Fama-French Factor Model

### Data Settings

In [2]:
import pandas as pd
import numpy as np
import pickle
data_path = 'simulation_data'

start_date = '2016-03-30'
end_date = '2025-03-20'
with open(f"{data_path}/KS200_MASK.pkl", 'rb') as f:
    mask_df:pd.DataFrame = pickle.load(f).ffill(axis=1)
    mask_df = mask_df.loc[:, start_date:end_date]
with open(f"{data_path}/Return.pkl", 'rb') as f:
    returns_df = pickle.load(f).ffill(axis=1) * 0.01
    returns_df = returns_df.loc[:, start_date:end_date]
with open(f"{data_path}/MarketCap.pkl", 'rb') as f:
    mc_df = pickle.load(f).ffill(axis=1)
    mc_df = mc_df.loc[:, start_date:end_date]
with open(f"{data_path}/ifrs-full_Equity.pkl", 'rb') as f:
    be_df = pickle.load(f).ffill(axis=1)
    be_df = be_df.loc[:, start_date:end_date]
with open(f"{data_path}/KOSPI_Close.pkl", 'rb') as f:
    kospi_close_df = pickle.load(f).ffill(axis=1)
    kospi_return_df = kospi_close_df.pct_change(axis=1)
    mkt_df = kospi_return_df.loc[:, start_date:end_date]
with open(f"{data_path}/rf_bond.pkl", 'rb') as f:
    rf_df = pickle.load(f).ffill(axis=1) * 0.01
    rf_df = rf_df.loc[:, start_date:end_date]
with open(f"{data_path}/corp_aa_bond.pkl", 'rb') as f:
    corp_aa_df = pickle.load(f).ffill(axis=1) * 0.01
    corp_aa_df = corp_aa_df.loc[:, start_date:end_date]
with open(f"{data_path}/corp_bb_bond.pkl", 'rb') as f:
    corp_bb_df = pickle.load(f).ffill(axis=1) * 0.01
    corp_bb_df = corp_bb_df.loc[:, start_date:end_date]
with open(f"{data_path}/gov10_bond.pkl", 'rb') as f:
    gov10_df = pickle.load(f).ffill(axis=1) * 0.01
    gov10_df = gov10_df.loc[:, start_date:end_date]
with open(f"{data_path}/gov3_bond.pkl", 'rb') as f:
    gov3_df = pickle.load(f).ffill(axis=1) * 0.01
    gov3_df = gov3_df.loc[:, start_date:end_date]
corp_df = (corp_aa_df + corp_bb_df) / 2

In [3]:
R     = returns_df
MC    = mc_df
BE    = be_df 
RF = rf_df / 252 # 일단위 reblancing 시행하므로 252로 나누어 줌.
GOV10   = gov10_df / 252
GOV3 = gov3_df / 252
CORP  = corp_df / 252
MKT   = mkt_df
mask  = mask_df.astype(bool)

dates = R.columns
tickers = R.index

In [4]:
port_rets = {f"S{i}B{j}":[] for i in range(1,6) for j in range(1,6)}

for dt in dates:
    members = mask[dt][mask[dt]].index
    if members.empty: 
        continue

    # size quintile breakpoints
    me = MC.loc[members, dt].dropna()
    sz_br = me.quantile([0.2,0.4,0.6,0.8]).values

    # BE/ME quintile breakpoints
    bm = (BE.loc[members, dt] / MC.loc[members, dt]).dropna()
    bm_br = bm.quantile([0.2,0.4,0.6,0.8]).values

    # 각 포트폴리오
    for i in range(1,6):
        size_idx = me[
            (me >  (sz_br[i-2] if i>1 else -np.inf)) &
            (me <= (sz_br[i-1] if i<5 else  np.inf))
        ].index
        for j in range(1,6):
            bm_idx = bm[
              (bm >  (bm_br[j-2] if j>1 else -np.inf)) &
              (bm <= (bm_br[j-1] if j<5 else  np.inf))
            ].index

            idx = size_idx.intersection(bm_idx)
            name = f"S{i}B{j}"
            if idx.empty:
                ret = np.nan
            else:
                w = MC.loc[idx, dt]
                w = w / w.sum()
                ret = (R.loc[idx, dt] * w).sum()
            port_rets[name].append((dt, ret))

# DataFrame으로 변환
port_df = pd.DataFrame({
    name: pd.Series(dict(port_rets[name]))
    for name in port_rets
}).sort_index(axis=1)
port_df = port_df.interpolate(method='linear', axis=0, limit_direction='both')


# ── 2) SMB, HML 계산 ──
# SMB: size quintile 1 평균 – quintile 5 평균
SMB = (port_df[[f"S1B{j}" for j in range(1,6)]].mean(axis=1) + port_df[[f"S2B{j}" for j in range(1,6)]].mean(axis=1)) \
        - (port_df[[f"S5B{j}" for j in range(1,6)]].mean(axis=1) + port_df[[f"S4B{j}" for j in range(1,6)]].mean(axis=1)) 

# HML: BE quintile 5 평균 – quintile 1 평균
HML = (port_df[[f"S{i}B5" for i in range(1,6)]].mean(axis=1) + port_df[[f"S{i}B4" for i in range(1,6)]].mean(axis=1))\
     - (port_df[[f"S{i}B1" for i in range(1,6)]].mean(axis=1) + port_df[[f"S{i}B2" for i in range(1,6)]].mean(axis=1))


factors = pd.DataFrame({
    'MKT_RF': MKT.iloc[0] - RF.iloc[0],
    'SMB':     SMB,
    'HML':     HML,
    'TERM':    GOV10.iloc[0]  - RF.iloc[0],
    'DEF':     CORP.iloc[0] - GOV3.iloc[0],
    'RF':      RF.iloc[0]
}, index=dates).dropna()

In [5]:
factors

,MKT_RF,SMB,HML,TERM,DEF,RF
2016-03-30,0.003560,-0.004864,-0.008255,0.000007,0.000138,0.000064
2016-03-31,-0.003206,0.012003,-0.004497,0.000007,0.000138,0.000064
2016-04-01,-0.011227,0.021868,-0.009723,0.000007,0.000138,0.000064
2016-04-04,0.002672,-0.002250,-0.021757,0.000006,0.000138,0.000064
2016-04-05,-0.008265,0.011213,-0.002196,0.000007,0.000138,0.000064
...,...,...,...,...,...,...
2025-03-14,-0.002941,0.001041,-0.004142,-0.000001,0.000137,0.000113
2025-03-17,0.017161,-0.012326,-0.007089,-0.000001,0.000137,0.000113
2025-03-18,0.000519,-0.009031,-0.000572,-0.000002,0.000137,0.000113
2025-03-19,0.006119,0.000945,0.010262,-0.000001,0.000136,0.000113


### Prior Settings
<img align="middle" src="images\bayesian_linear_reg.png" height="400" />

<img align="middle" src="images\g-prior_settings.png" height="400" />

<img align="middle" src="images\model_selection_settings.png" height="400" />

<img align="middle" src="images\steps_for_MCMC.png" height="400" />

My model for Bayesian FF

Y = z_0*beta_0 + z_1*beta_1 * MKT_RF + z_2*beta_2 * SMB + z_3*beta_3 * HML + z_4*beta_4 * TERM + z_5*beta_5 * DEF + e

In [ ]:
import scipy.stats as stats
from numpy.linalg import pinv
# drop the last column (RF)
y = np.array(port_df.subtract(factors['RF'], axis=0))
X = np.array(factors.iloc[:, :-1])
X = np.vstack([np.ones(X.shape[0]), X.T]).T

# prior parameters
n = X.shape[0] # inherit BIC’s consistency and avoid the Bartlett/Lindley paradox
num_features = X.shape[1] # number of features

nu_0 = X.shape[0] + 2 # low confidence of the prior
sigma_0 = 1

z_0 = np.array([1, 1, 1, 1, 0, 0]) # only MKT, SMB, HML are included
X_z = X @ np.diag(z_0)

SSR_g = y.T@(np.eye(n) - n/(n+1)*X_z@pinv(X_z.T@X_z)@X_z.T)@y # 25 by 25 matrix


In [19]:
SSR_g.shape

(25, 25)